# Part 2b: Add Validation

Load and process patient data with BMI calculations.

**Your task:** Add schema and bounds validation to catch data quality issues early.

---

## Load data

In [12]:
import pandas as pd

# TODO: Define a function validate_schema(df, required_columns) that:
#       - Checks if all required columns are present
#       - Raises ValueError with list of missing columns if any are missing
def validate_schema(df, required_columns):
    missing_columns = set(required_columns) - set(df.columns)
    if missing_columns:
        raise ValueError(f"Missing columns: {missing_columns}")

# TODO: Define a function validate_bounds(df, bounds_dict) that:
#       - For each column in bounds_dict, check if values are within (min, max)
#       - Use df[col].between(min, max) to find out-of-bounds values
#       - Raises ValueError showing patient_id and value for any out-of-bounds rows
def validate_bounds(df, bounds_dict):
    for col, (min_val, max_val) in bounds_dict.items():
        out_of_bounds = ~df[col].between(min_val, max_val)
        if out_of_bounds.any():
            bad_rows = df[out_of_bounds][["patient_id", col]]
            raise ValueError(f"{col} out of bounds:\n{bad_rows}")

df = pd.read_csv("data/patient_intake.csv")

# TODO: Call validate_schema() to check for required columns:
#       ["patient_id", "weight_kg", "height_cm", "age"]
required_cols = ["patient_id", "weight_kg", "height_cm", "age"]
validate_schema(df, required_cols)


# TODO: Call validate_bounds() with bounds:
#       weight_kg: (30, 250)
#       height_cm: (120, 230)
#       age: (0, 110)
bounds = {
    "weight_kg": (30, 250),
    "height_cm": (120, 230),
    "age": (0, 110)
}
validate_bounds(df, bounds)
df.head()

,patient_id,first_name,last_name,weight_kg,height_cm,age,sex
0,P001,Mark,Johnson,91.5,177,46,M
1,P002,Donald,Walker,80.5,164,29,M
2,P003,Nancy,Rhodes,74.3,163,47,F
3,P004,Steven,Miller,64.4,171,71,M
4,P005,Javier,Johnson,72.8,178,18,M


---

## Calculate BMI

In [17]:
df["height_m"] = df["height_cm"] / 100
df["bmi"] = df["weight_kg"] / (df["height_m"] ** 2)
df["bmi"] = df["bmi"].round(1)

df[["patient_id", "weight_kg", "height_cm", "bmi"]].head()

,patient_id,weight_kg,height_cm,bmi
0,P001,91.5,177,29.2
1,P002,80.5,164,29.9
2,P003,74.3,163,28.0
3,P004,64.4,171,22.0
4,P005,72.8,178,23.0


---

## Categorize BMI

In [18]:
df["bmi_category"] = pd.cut(
    df["bmi"],
    bins=[0, 18.5, 25, 30, float("inf")],
    labels=["Underweight", "Normal", "Overweight", "Obese"],
    right=False
)

df[["patient_id", "bmi", "bmi_category"]].head()

,patient_id,bmi,bmi_category
0,P001,29.2,Overweight
1,P002,29.9,Overweight
2,P003,28.0,Overweight
3,P004,22.0,Normal
4,P005,23.0,Normal


---

## Summary statistics

In [19]:
summary = df.groupby("bmi_category")["patient_id"].count()
print("\nBMI category distribution:")
print(summary)

high_risk = df[df["bmi"] > 30]
print(f"\nHigh-risk patients (BMI > 30): {len(high_risk)}")


BMI category distribution:
bmi_category
Underweight     0
Normal         15
Overweight     21
Obese          14
Name: patient_id, dtype: int64

High-risk patients (BMI > 30): 14


/var/folders/49/w0jyt4j1499dvzs2fxpfr6r00000gn/T/ipykernel_59682/3669141136.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  summary = df.groupby("bmi_category")["patient_id"].count()
